In [ ]:
from pathlib import Path

import numpy as np
from superjson import json

from data import Flickr30k
from utils import show_image

loader = Flickr30k(split="val")

In [ ]:
def save_annotations(
    image_id: int,
    caption_id: int,
    original_sentence: str,
    translated_sentence: str,
    marker: str,
    folder: Path = Path("./translations"),
):
    obj = {
        "markers": [marker],
        "original_sentence": original_sentence,
        "translated_sentences": [translated_sentence],
    }
    translation_file: Path = folder / image_id / f"{caption_id}.json"
    if translation_file.is_file():
        data = json.load(str(translation_file), verbose=False)
        data["markers"].append(marker)
        data["translated_sentences"].append(translated_sentence)
        obj = data
    json.dump(obj, str(translation_file), indent=2, overwrite=True, verbose=False)

In [ ]:
def translate_caption(caption):
    print(caption)
    user_input = input("Translation: ")
    if not user_input:
        print("Empty string is not allowed.")
        return None
    if user_input[0] == ">":
        command = user_input[1:]
        if command == "skip":
            return "SKIP"

    return user_input


for i, (image, captions) in enumerate(loader):
    show_image(np.asarray(image))
    for j, caption in enumerate(captions):
        result = None
        while result is None:
            result = translate_caption(caption)
        save_annotations(i, j, caption, *result)
    break
